# Целевые переменные

- Целевые функции: realized volatility horizon, spike_flag, quantiles
- Анализ распределения target'ов, сохранение меток для обучения моделей

Критически важно для ML-задачи правильное создание и смещение целевых переменных!


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

print("✅ Библиотеки загружены")


## Функции создания целевых переменных


In [ ]:
def create_realized_vol_target(returns, horizon=5):
    """
    Целевая переменная: реализованная волатильность на горизонте
    Сдвиг на будущее для предсказания
    """
    # Рассчитываем стандартное отклонение на следующие 'horizon' дней
    target = returns.rolling(window=horizon).std().shift(-horizon) * np.sqrt(252)
    return target


def create_spike_flag(returns, threshold=2.0, window=20):
    """
    Бинарная целевая: будет ли всплеск волатильности
    1 = превышение порога, 0 = нормальное состояние
    """
    # Расчет z-score на будущий период
    future_returns = returns.shift(-1)
    ma = returns.rolling(window=window).mean()
    std = returns.rolling(window=window).std()
    z_score = np.abs((future_returns - ma) / std)
    
    spike = (z_score > threshold).astype(int)
    return spike


def create_quantile_targets(returns, horizon=5, quantiles=[0.16, 0.50, 0.84]):
    """
    Квантили распределения будущих доходностей
    Используется для quantile regression
    """
    targets = {}
    for q in quantiles:
        # Вычисляем квантиль на скользящем окне будущих данных
        target = returns.rolling(window=horizon).quantile(q).shift(-horizon)
        targets[f"quantile_{int(q*100)}"] = target
    
    return pd.DataFrame(targets)


def create_directional_target(returns, horizon=1):
    """
    Направление движения: 1 = вверх, 0 = вниз
    Для классификации
    """
    future_return = returns.shift(-horizon)
    direction = (future_return > 0).astype(int)
    return direction


def create_all_targets(df, horizons=[1, 5, 10]):
    """
    Создание всех целевых переменных для DataFrame
    """
    df = df.copy()
    
    # Целевая волатильность на разных горизонтах
    for h in horizons:
        df[f"target_vol_{h}d"] = create_realized_vol_target(df["log_return"], horizon=h)
    
    # Спайк флаг
    df["target_spike"] = create_spike_flag(df["log_return"], threshold=2.0, window=20)
    
    # Квантили
    quantile_df = create_quantile_targets(df["log_return"], horizon=5, quantiles=[0.16, 0.50, 0.84])
    df = pd.concat([df, quantile_df], axis=1)
    
    # Направление
    df["target_direction"] = create_directional_target(df["log_return"], horizon=1)
    
    return df

print("✅ Функции целевых переменных загружены")


## Загрузка данных


In [ ]:
# Получаем текущую рабочую директорию
current_dir = Path.cwd()
print(f"Текущая директория: {current_dir}")

# Строим путь к данным относительно текущей директории
DATA_DIR = current_dir.parent.parent / "ML" / "data" / "processed"
OUTPUT_DIR = Path("data") / "features"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
print(f"Путь к данным: {DATA_DIR}")

# Проверяем существование пути
if DATA_DIR.exists():
    print("Директория найдена!")
else:
    print("Директория не найдена, проверьте путь")

# Список всех доступных тикеров включая индекс MOEX
TICKERS = [
    "AFKS", "AFLT", "ALRS", "BELU", "BSPB", "CHMF", "FIVE", "GAZP", "GMKN", "HYDR",
    "IMOEX",  # Индекс Мосбиржи
    "IRAO", "LENT", "LKOH", "MAGN", "MGNT", "MTSS", "NLMK", "NVTK", "OZON",
    "PIKK", "PLZL", "ROSN", "RTKM", "SBER", "SNGS", "TATN", "TCSG", "VKCO", "VTBR", "YNDX"
]

print(f"✅ Загрузим {len(TICKERS)} тикеров")
print(f"Тикеры: {TICKERS}")


## Расчет для всех тикеров


In [ ]:
# Загружаем и обрабатываем все тикеры
results = {}
horizons = [1, 5, 10]

for ticker in TICKERS:
    try:
        print(f"🔄 Обработка {ticker}...", end=" ")
        
        # 1. Загрузка данных
        input_path = DATA_DIR / f"{ticker}_ohlcv_returns.parquet"
        if not input_path.exists():
            print(f"⚠️ Файл не найден: {input_path}")
            continue
        
        df = pd.read_parquet(input_path)
        
        # 2. Создание целевых переменных
        df = create_all_targets(df, horizons=horizons)
        
        # 3. Подсчет целевых столбцов
        target_cols = [col for col in df.columns if col.startswith("target_") or col.startswith("quantile_")]
        
        # 4. Сохранение в словарь результатов
        results[ticker] = df
        print(f"✅ Готово ({len(df)} строк, {len(target_cols)} целевых)")
        
    except Exception as e:
        print(f"\n❌ Ошибка при обработке {ticker}: {e}")

print("-" * 50)
print(f"🎉 Обработка завершена. Успешно обработано: {len(results)} из {len(TICKERS)}")


## Пример и диагностика


In [ ]:
# Демонстрация на примере SBER
sample_ticker = "SBER"
if sample_ticker in results:
    df_sample = results[sample_ticker]
    
    # Целевые столбцы
    target_cols = [col for col in df_sample.columns if col.startswith("target_") or col.startswith("quantile_")]
    print(f"Целевые столбцы: {target_cols}")
    
    # Статистика целевой волатильности
    print(f"\n=== Статистика целевой волатильности (5 дней) ===")
    print(df_sample["target_vol_5d"].describe())
    
    # Распределение spike flag
    print(f"\n=== Распределение spike flag ===")
    print(df_sample["target_spike"].value_counts())
    spike_rate = df_sample["target_spike"].mean()
    print(f"Доля всплесков: {spike_rate:.2%}")
    
    # Распределение direction
    print(f"\n=== Распределение direction ===")
    print(df_sample["target_direction"].value_counts())
    growth_rate = df_sample["target_direction"].mean()
    print(f"Доля роста: {growth_rate:.2%}")
    
    # Квантили
    print(f"\n=== Квантили ===")
    print(df_sample[["quantile_16", "quantile_50", "quantile_84"]].describe())
    
    # Пример данных
    print(f"\n=== Последние 10 записей ===")
    display_cols = ["date", "close", "log_return", "target_vol_5d", "target_spike", "target_direction"]
    print(df_sample[display_cols].tail(10))
else:
    print(f"Тикер {sample_ticker} не найден в результатах")


## Сохранение результатов


In [ ]:
# Сохранение всех результатов
for ticker, df in results.items():
    output_path = OUTPUT_DIR / f"{ticker}_with_targets.parquet"
    df.to_parquet(output_path, index=False)
    print(f"✅ Сохранено: {output_path}")

print(f"\n🎉 Все данные сохранены в {OUTPUT_DIR}")

# Итоговая статистика
if results:
    sample_df = list(results.values())[0]
    target_cols = [col for col in sample_df.columns if col.startswith("target_") or col.startswith("quantile_")]
    print(f"\n📊 Итого:")
    print(f"   Обработано тикеров: {len(results)}")
    print(f"   Целевых переменных: {len(target_cols)}")
    print(f"   Всего столбцов: {len(sample_df.columns)}")
